In [5]:
import pandas as pd

train = pd.read_csv("../data/movie/labeledTrainData.tsv",
                    header = 0,
                    delimiter="\t",
                    quoting=3)
test = pd.read_csv("../data/movie/testData.tsv",
                    header = 0,
                    delimiter="\t",
                    quoting=3)
unlabeld_train = pd.read_csv("../data/movie/unlabeledTrainData.tsv",
                    header = 0,
                    delimiter="\t",
                    quoting=3)

In [6]:
display(test.head(1))
display(train.head(1))
display(unlabeld_train.head(1))

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."


,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."


In [10]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False):
    # 1. HTML 태그 지우기
    review_text = BeautifulSoup(review).get_text()
    # 2. 알파벳 빼고 다 지움
    review_text = re.sub('[^a-zA-Z]', '', review_text)
    # 3. 소문자로 바꾸고 단어 분리
    words = review_text.lower().split()
    # 4. 정지단어 제거
    if remove_stopwords:
        stops = set(stopwordswords.word("english"))
        words = [w for w in words if not w in stops]
    
    return (words)

In [14]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentences in raw_sentences:
        if len(raw_sentences) >0:
            sentences.append(review_to_wordlist(raw_sentences, remove_stopwords))
    return sentences

In [20]:
sentences = []

for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)
    
for reveiw in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

C:\ProgramData\Anaconda3\envs\gpu_env\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\envs\gpu_env\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\envs\gpu_env\lib\site-packages\bs4\__init__.py:335: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [21]:
print(len(sentences))

491551
